In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
from IPython.display import display
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv, split_train_test_data

2023-07-03 03:03:14.343455: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 03:03:14.497412: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 03:03:14.955285: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 03:03:14.956797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 03:03:15.951564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
#mio
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from IPython.display import display
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv, split_train_test_data
def create_sequences(data, sequence_length, output_length):
    X, Y = [], []
    shape = data.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        rows, cols = data.shape[0], 1
        data = data.reshape(rows,cols)
    else:
        rows, cols = data.shape
    # Generar los arreglo
    for i in range(rows- sequence_length):
        X.append(data[i: i + sequence_length, 0: cols])
        Y.append(data[i + sequence_length : i + sequence_length + output_length, -1].reshape(output_length,1))
    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)
    return X, Y
def train_test_split(data):
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]
    return train_data, test_data
def escalar_dataset(data_input):
    NFEATS = data_input['x_tr'].shape[2]

    # Generar listado con "scalers"
    # scalers = [MinMaxScaler(feature_range=(0,1)) for i in range(NFEATS)] #feature_rangetuple (min, max), default=(0, 1)
    scalers = [StandardScaler() for i in range(NFEATS)] #feature_rangetuple (min, max), default=(0, 1)

    # Arreglos que contendrán los datasets escalados
    x_tr_s = np.zeros(data_input['x_tr'].shape)
    # x_vl_s = np.zeros(data_input['x_vl'].shape)
    x_ts_s = np.zeros(data_input['x_ts'].shape)
    y_tr_s = np.zeros(data_input['y_tr'].shape)
    # y_vl_s = np.zeros(data_input['y_vl'].shape)
    y_ts_s = np.zeros(data_input['y_ts'].shape)

    # Escalamiento: se usarán los min/max del set de entrenamiento para
    # escalar la totalidad de los datasets
    
    # Escalamiento Xs
    for i in range(NFEATS):
        x_tr_s[:,:,i] = scalers[i].fit_transform(data_input['x_tr'][:,:,i])
        # x_vl_s[:,:,i] = scalers[i].transform(x_vl[:,:,i])
        x_ts_s[:,:,i] = scalers[i].transform(data_input['x_ts'][:,:,i])
    
    # Escalamiento Ys
    y_tr_s[:,:,0] = scalers[-1].fit_transform(data_input['y_tr'][:,:,0])
    # y_vl_s[:,:,0] = scalers[-1].transform(y_vl[:,:,0])
    y_ts_s[:,:,0] = scalers[-1].transform(data_input['y_ts'][:,:,0])

    # Conformar ` de salida
    data_scaled = {
        'x_tr_s': x_tr_s, 'y_tr_s': y_tr_s,
        # 'x_vl_s': x_vl_s, 'y_vl_s': y_vl_s,
        'x_ts_s': x_ts_s, 'y_ts_s': y_ts_s,
    }

    return data_scaled, scalers[0]

def main():
    path_load = "../../data/"
    file_name = "processed/df_time_monthly.csv"
    full_path = path_load + file_name
    data = load_dataframe_from_csv(full_path)
    data.index.freq = 'M'
    #
    INPUT_LENGTH = 3
    OUTPUT_LENGTH = 1
    train_data, test_data =train_test_split(data['cost'])
    # Imprimir en pantalla el tamaño de cada subset
    print(f'Tamaño set de entrenamiento: {train_data.shape}')
    print(f'Tamaño set de prueba: {test_data.shape}')

    x_tr, y_tr = create_sequences(train_data.values, INPUT_LENGTH, OUTPUT_LENGTH)
    x_ts, y_ts = create_sequences(test_data.values, INPUT_LENGTH, OUTPUT_LENGTH)
    print('Tamaños entrada (BATCHES x INPUT_LENGTH x FEATURES) y de salida (BATCHES x OUTPUT_LENGTH x FEATURES)')
    print(f'Set de entrenamiento - x_tr: {x_tr.shape}, y_tr: {y_tr.shape}')
    print(f'Set de prueba - x_ts: {x_ts.shape}, y_ts: {y_ts.shape}')

    # Preprocesamiento de los datos
    data_in = {
    'x_tr': x_tr, 'y_tr': y_tr,
    # 'x_vl': x_vl, 'y_vl': y_vl,
    'x_ts': x_ts, 'y_ts': y_ts,
    }
    data_s, scaler = escalar_dataset(data_in)
    # Extraer subsets escalados
    x_tr_s, y_tr_s = data_s['x_tr_s'], data_s['y_tr_s']
    x_ts_s, y_ts_s = data_s['x_ts_s'], data_s['y_ts_s']
    
    # Verificación
    print(f'Min x_tr/x_ts sin escalamiento: {x_tr.min()}/{x_ts.min()}')
    print(f'Min x_tr/x_ts con escalamiento: {x_tr_s.min()}/{x_ts_s.min()}')
    
    print(f'\nMin y_tr/y_ts sin escalamiento: {y_tr.min()}/{y_ts.min()}')
    print(f'Min y_tr/y_ts con escalamiento: {y_tr_s.min()}/{y_ts_s.min()}')
    
    print(f'\nMax x_tr/x_ts sin escalamiento: {x_tr.max()}/{x_ts.max()}')
    print(f'Max x_tr/x_ts con escalamiento: {x_tr_s.max()}/{x_ts_s.max()}')
    
    print(f'\nMax y_tr/y_ts sin escalamiento: {y_tr.max()}/{y_ts.max()}')
    print(f'Max y_tr/y_ts con escalamiento: {y_tr_s.max()}/{y_ts_s.max()}')

    # Ajustar parámetros para reproducibilidad del entrenamiento
    tf.random.set_seed(123)
    tf.config.experimental.enable_op_determinism()
    
    # El modelo
    N_UNITS = 64 # Tamaño del estado oculto (h) y de la celda de memoria (c)
    INPUT_SHAPE = (x_tr_s.shape[1], x_tr_s.shape[2]) # 3 (meses) x 1 (feature)
    
    modelo = Sequential()
    modelo.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE))
    modelo.add(Dense(OUTPUT_LENGTH, activation='linear'))

    # Compilación
    optimizador = RMSprop(learning_rate=5e-5)
    modelo.compile(
        optimizer = optimizador,
        loss="mean_squared_error",
    )
    
    # Entrenamiento (aproximadamente 1 min usando GPU)
    EPOCHS = 80 # Hiperparámetro
    BATCH_SIZE = 256 # Hiperparámetro
    historia = modelo.fit(
        x = x_tr_s,
        y = y_tr_s,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        # validation_data = (x_vl_s, y_vl_s),
        verbose=2
    )
    rmse_tr = modelo.evaluate(x=x_tr_s, y=y_tr_s, verbose=0)
    # rmse_vl = modelo.evaluate(x=x_vl_s, y=y_vl_s, verbose=0)
    rmse_ts = modelo.evaluate(x=x_ts_s, y=y_ts_s, verbose=0)

    # Imprimir resultados en pantalla
    print('Comparativo desempeños:')
    print(f'  RMSE train:\t {rmse_tr:.3f}')
    # print(f'  RMSE val:\t {rmse_vl:.3f}')
    print(f'  RMSE test:\t {rmse_ts:.3f}')

    
if __name__ == "__main__":
    main()


Tamaño set de entrenamiento: (36,)
Tamaño set de prueba: (10,)
Tamaños entrada (BATCHES x INPUT_LENGTH x FEATURES) y de salida (BATCHES x OUTPUT_LENGTH x FEATURES)
Set de entrenamiento - x_tr: (33, 3, 1), y_tr: (33, 1, 1)
Set de prueba - x_ts: (7, 3, 1), y_ts: (7, 1, 1)
Min x_tr/x_ts sin escalamiento: 645200/2505194
Min x_tr/x_ts con escalamiento: -1.3069970656643133/0.4012460828971006

Min y_tr/y_ts sin escalamiento: 645200/2505194
Min y_tr/y_ts con escalamiento: -1.3303845156102565/0.33403438617066356

Max x_tr/x_ts sin escalamiento: 4345208/5296946
Max x_tr/x_ts con escalamiento: 2.224455437243326/3.421009231253509

Max y_tr/y_ts sin escalamiento: 4345208/3359652
Max y_tr/y_ts con escalamiento: 1.9805741498158784/1.0986476300724834
Epoch 1/80


2023-07-03 03:03:18.068341: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-03 03:03:18.070031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-03 03:03:18.070892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 - 1s - loss: 0.8600 - 1s/epoch - 1s/step
Epoch 2/80
1/1 - 0s - loss: 0.8571 - 4ms/epoch - 4ms/step
Epoch 3/80
1/1 - 0s - loss: 0.8549 - 5ms/epoch - 5ms/step
Epoch 4/80
1/1 - 0s - loss: 0.8531 - 4ms/epoch - 4ms/step
Epoch 5/80
1/1 - 0s - loss: 0.8514 - 3ms/epoch - 3ms/step
Epoch 6/80
1/1 - 0s - loss: 0.8500 - 4ms/epoch - 4ms/step
Epoch 7/80
1/1 - 0s - loss: 0.8486 - 4ms/epoch - 4ms/step
Epoch 8/80
1/1 - 0s - loss: 0.8473 - 3ms/epoch - 3ms/step
Epoch 9/80
1/1 - 0s - loss: 0.8460 - 3ms/epoch - 3ms/step
Epoch 10/80
1/1 - 0s - loss: 0.8448 - 6ms/epoch - 6ms/step
Epoch 11/80
1/1 - 0s - loss: 0.8436 - 9ms/epoch - 9ms/step
Epoch 12/80
1/1 - 0s - loss: 0.8425 - 10ms/epoch - 10ms/step
Epoch 13/80
1/1 - 0s - loss: 0.8414 - 4ms/epoch - 4ms/step
Epoch 14/80
1/1 - 0s - loss: 0.8403 - 4ms/epoch - 4ms/step
Epoch 15/80
1/1 - 0s - loss: 0.8392 - 4ms/epoch - 4ms/step
Epoch 16/80
1/1 - 0s - loss: 0.8381 - 4ms/epoch - 4ms/step
Epoch 17/80
1/1 - 0s - loss: 0.8371 - 4ms/epoch - 4ms/step
Epoch 18/80
1/1 -

2023-07-03 03:03:19.881438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [33,3,1]
	 [[{{node Placeholder/_10}}]]
2023-07-03 03:03:19.881655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [33,1,1]
	 [[{{node Placeholder/_11}}]]
2023-07-03 03:03:20.038289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split

Comparativo desempeños:
  RMSE train:	 0.778
  RMSE test:	 0.311


2023-07-03 03:03:20.176872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [7,1,1]
	 [[{{node Placeholder/_11}}]]
2023-07-03 03:03:20.177063: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [7,1,1]
	 [[{{node Placeholder/_11}}]]
